# Zero-shot mutant prediction For creatinase

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["http_proxy"] = "http://g5:15777"
os.environ["https_proxy"] = "http://g5:15777"

def read_seq(seq_file):
    for record in SeqIO.parse(seq_file, "fasta"):
        return str(record.seq)

wild_type = f"../example_data/expr/creatinase.fasta"
sequence = read_seq(wild_type)

all_mutants = []
for i, wt in enumerate(sequence):
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        if aa != wt:
            mutant = f"{wt}{i+1}{aa}"
            all_mutants.append(mutant)
df = pd.DataFrame({"mutant": all_mutants})

In [2]:
base_model_path = "AI4Protein/creatinase_model_"

for i in range(0, 5):
    model_path = f"{base_model_path}{i}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForMaskedLM.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained("AI4Protein/Prime_690M", trust_remote_code=True)
    model.eval()
    model = model.to(device)

    tokenized = tokenizer(sequence, return_tensors="pt").to(device)
    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits[0, 1:-1, :].log_softmax(dim=-1)
    
    scores = []
    for mutant in tqdm(df["mutant"]):
        score = 0
        for sub_mutant in mutant.split(":"):
            wt, idx, mt = sub_mutant[0], int(sub_mutant[1:-1]) - 1, sub_mutant[-1]
            score += (logits[idx, tokenizer.get_vocab()[mt]] - logits[idx, tokenizer.get_vocab()[wt]]).item()
        scores.append(score)
    df[f"model_{i}"] = scores
    print(f"Model {i} done")

  0%|          | 0/7676 [00:00<?, ?it/s]

Model 0 done


  0%|          | 0/7676 [00:00<?, ?it/s]

Model 1 done


  0%|          | 0/7676 [00:00<?, ?it/s]

Model 2 done


  0%|          | 0/7676 [00:00<?, ?it/s]

Model 3 done


  0%|          | 0/7676 [00:00<?, ?it/s]

Model 4 done


In [3]:
aggregate_score = 0
for i in range(0, 5):
    # min-max
    df[f"model_{i}"] = (df[f"model_{i}"] - df[f"model_{i}"].min()) / (df[f"model_{i}"].max() - df[f"model_{i}"].min())
    aggregate_score += df[f"model_{i}"]
df["aggregate"] = aggregate_score
df = df.sort_values("aggregate", ascending=False)

In [4]:
df.head(50)

,mutant,model_0,model_1,model_2,model_3,model_4,aggregate
351,S19L,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000
7444,G392T,0.999226,0.999226,0.999231,0.999226,0.999224,4.996134
3408,A180K,0.979993,0.979992,0.979989,0.979991,0.979993,4.899959
1399,H74Q,0.975267,0.975267,0.975269,0.975268,0.975265,4.876337
3062,L162E,0.965547,0.965547,0.965543,0.965547,0.965545,4.827728
1970,R104Q,0.959104,0.959101,0.959097,0.959101,0.959098,4.795501
1978,R105D,0.957739,0.957739,0.957735,0.957740,0.957737,4.788689
2866,Q151V,0.952905,0.952906,0.952909,0.952908,0.952907,4.764536
1709,W90Y,0.952360,0.952362,0.952367,0.952362,0.952360,4.761810
5880,Y310L,0.952347,0.952348,0.952354,0.952348,0.952347,4.761744
